In [159]:
import json
import pandas as pd
import os
import sqlite3
from itertools import islice
from sklearn.metrics.pairwise import cosine_similarity

from enum import Enum
from typing import List
from pydantic import BaseModel, Field

#!pip install openai -U
from openai import OpenAI
client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))
model = "gpt-4o-2024-08-06"

In [15]:
# Get human emotions    
 
# Define the Pydantic model for the API response
class EmotionsResponse(BaseModel):
    Characteristics: List[str] = Field(None, description="List of non-redundant human emotions.")

def get_emotions(model: str) -> List[str]:
    """Gets a list of 50 unique and non-redundant human emotions using the specified gpt model."""
    
    # Define system and user prompts
    system_prompt = "Find 50 different, exclusive and unique human emotions. "\
    "For example, pick joy or happiness, pick Shame or Embarrassment, pick Envy or Jealousy, "\
    "pick Hate or disgust or hatered or Resentment. "\

    user_prompt = "Select 50 different and unique human emotions."

    try:
        #Call the API to get the completion
        completion = client.beta.chat.completions.parse(
            model= model,
            messages=[
                {"role": "system", "content": "Be a helpful assistant."},
                {"role": "system", "content": system_prompt},
                {"role": "system", "content": "make sure to include either joy or happiness, not both."},
                {"role": "system", "content": "make sure to include either Shame or Embarrassment, not both"},
                {"role": "system", "content": "make sure to include either Envy or Jealousy, not both"},
                {"role": "system", "content": "make sure to include either Hate or disgust or hatered or Resentment"},
                {"role": "system", "content": "Check again to remove redundant emotions. I only want unique emotions."},
                {"role": "user", "content": user_prompt}
            ],
            response_format=EmotionsResponse
        )

        #output returns in the defined pydantic style
        output = completion.choices[0].message.parsed
        return output.json()
    
    except Exception as e:
        # Handle exceptions such as API errors, etc
        print(f"An error occurred: {e}")
        return json.dumps({})

# Example usage
emotions = get_emotions(model= model)

In [16]:
#Get 100 best selling American clothing brands 

# Define the Pydantic model for the API response
class BrandsResponse(BaseModel):
    Brands: List[str] = Field(None, description="Brands as a list of strings.")

def get_brands(model: str) -> List[str]:
    """Get 100 best selling American clothing brands using the specified gpt model."""

    try:
        #Call the API to get the completion
        completion = client.beta.chat.completions.parse(
            model= model,
            messages=[
                {"role": "system", "content": "Be a helpful assistant."},
                {"role": "system", "content": "Find 100 non-redundant best selling American clothing brands."},
                {"role": "system", "content": "DONT MAKE ANY MISTAKES, check if you did any."},
                {"role": "user", "content": "Give me 100 best selling American clothing brands."}
            ],
            response_format=BrandsResponse
        )

        #output returns in the defined pydantic style
        output = completion.choices[0].message.parsed
        return output.json()
    
    except Exception as e:
        # Handle exceptions such as API errors, etc
        print(f"An error occurred: {e}")
        return json.dumps({})

# Example usage
brands = get_brands(model= model)

In [17]:
emotions_ls = list(json.loads(emotions).values())[0]
brands_ls = list(json.loads(brands).values())[0]

In [38]:
# Embedding brand in emotions space: Get association scores between an input and list of emotions

Characteristic = Enum('Characteristic', dict([(emotion, emotion) for emotion in emotions_ls]))

class EmotionalAssociationScore(BaseModel):
    emotion: Characteristic
    score: float

class EmotionalAssociationScores(BaseModel):
    associations: List[EmotionalAssociationScore] = Field(description="A list of emotions and associated scores")

def emotional_association_scores(
        thing, 
        model,
        emotions
    ):
    
    prompt = f"Assign emotional association scores between {0} and {len(emotions)} for the provided thing. "\
    "Assign a score for each of the following emotions. Briefly, explain the reason behind the association score."\
    "Ensure the scores reflect the association strength for the specified thing. "\
    "Thing: "\
    f"{thing}"
            
    completion = client.beta.chat.completions.parse(
        model = model,
        messages=[
            {"role": "system", "content": "Be a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        response_format=EmotionalAssociationScores,
    )
    #output returns in the defined pydantic style
    output = completion.choices[0].message.parsed
    return thing, output.json()

In [ ]:
#not using this for the moment
# #Embedding brands in emotions space: 
# # tried nested prompt but decided to go with one prompt and a list comprehension
# emotions= emotions_ls
# associations_brands = [emotional_association_scores(thing, model, emotions) for thing in brands_ls[:3]]


In [ ]:
def get_df(thing, model, emotions):
    gpt = emotional_association_scores(thing, model, emotions)
    data = list(json.loads(gpt[1]).values())[0]
    df = pd.DataFrame(data)
    df.rename(columns = {'score': gpt[0]}, inplace=True)
    df.set_index('emotion', inplace=True)
    return df

def get_dfs(things_ls, model, emotions):
    merged_df = pd.DataFrame()
    for thing in things_ls:
        new_df = get_df(thing, model, emotions)
        if merged_df.empty:
            merged_df = new_df
        else:
            merged_df = pd.merge(merged_df, new_df, left_index=True, right_index=True, how='outer')
    return merged_df


things_ls = brands_ls
dfs = get_dfs(things_ls, model, emotions)
# Drop columns with NaN values
dfs_cleaned = dfs.dropna(axis=1)

dfs_cleaned 

In [ ]:
# # Set pandas to display all rows and columns
# pd.set_option('display.max_rows', None)  # Show all rows
# pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.width', None)  # Adjust display width to prevent column cutting
# pd.set_option('display.max_colwidth', None)  # Show full content in columns
# dfs.isna().sum()


In [ ]:
# #Confirmed no need to l2 norm vectors for sklearn's cosine similarity:
# # Define your original vectors
# A = np.array([[2, 3]])
# B = np.array([[5, 4]])

# # Calculate cosine similarity without normalization
# cosine_sim_without_norm = cosine_similarity(A, B)

# # L2 normalize the vectors
# A_normalized = A / np.linalg.norm(A)
# B_normalized = B / np.linalg.norm(B)

# # Calculate cosine similarity with normalization
# cosine_sim_with_norm = cosine_similarity(A_normalized, B_normalized)

# # Print the outputs
# print("Cosine Similarity without normalization:")
# print(cosine_sim_without_norm[0][0])  # Output from unnormalized vectors

# print("\nCosine Similarity with normalization:")
# print(cosine_sim_with_norm[0][0])      # Output from normalized vectors
# cosine_sim_without_norm[0][0]==cosine_sim_with_norm[0][0]

In [ ]:

def get_similarity(df, dfs):
    similarities = dict()

    # Reshape Series to 2D array (required by cosine_similarity)
    s1 = df.values.reshape(1, -1)

    for col in list(dfs.columns):
        # Reshape
        s2= dfs[col].values.reshape(1, -1)

        cosine_sim = cosine_similarity(s1, s2)
        similarities[col]= cosine_sim[0][0]

    sorted_dict = dict(sorted(similarities.items(), key=lambda item: item[1], reverse = True))

    # Get the top 3 (highest similarity)
    top_3 = list(dict(islice(sorted_dict.items(), 3)).keys())

    return top_3

get_similarity(df, dfs_cleaned)


In [24]:
#The cosine similarity ranges from -1 to 1, where:
#1 indicates identical vectors (i.e., vectors point in the same direction).
#0 indicates orthogonality (i.e., vectors are at a 90-degree angle to each other, no similarity).
#-1 indicates opposite directions (i.e., vectors point in exactly opposite directions).
#represents similarity between feature vectors, quantifying similarity between two vectors based on their direction, 
# irrespective of their magnitude.

#embeddings happen in a much smaller space of emotions as oppossed to ordinary, more common embeddings in a large space as more commonly done with openai api (read)

In [ ]:
# a method
#embedding dimension is emotions
#talk about options
#get the brands, go through 50 emotins at a time
#cosine: normalize first: l2 norm = 1
#give instructions on readme on where key goes 
#first have everything in pandas df, then think about database
# one module or package w 1 .py 
#adaptors that take in pydantic datatypes and will make into sql
#argparse


In [ ]:
# #to check emotion redundancy by looking at example groups 
#[i for i in list(json.loads(emotions).values())[0] if i in ['Joy', 'Happiness', 'Shame', 'Embarrassment', 'Envy', 'Jealousy' , 'Hate', 'disgust', 'hatered', 'Resentment']]

In [ ]:
#test0

#Retrieve emotions from datbase or through openAI API
# if os.path.exists('emotions.json'):
#     with open('emotions.json', 'r') as f:
#         emotions_json = json.load(f)
# else:
#     emotions_json = get_emotions(model, api_key)

#test sqlite
# with sqlite3.connect(os.path.abspath('database.db')) as conn:
#     # Write the DataFrame to the database
#     df.to_sql('mytable', conn, if_exists='replace', index=False)
#     #cursor = conn.cursor()
#     #cursor.execute('SELECT SQLITE_VERSION()')
#     #data = cursor.fetchone()
#     #print('SQLite version:', data)

# query = "SELECT * FROM mytable"
# with sqlite3.connect(os.path.abspath('database.db')) as conn:
#     df_test= pd.read_sql_query(query, conn)


In [224]:
#test0
#test

class EmotionsResponse(BaseModel):
    #None as default if value not provided
    Emotions: List[str] = Field(None, description="List of non-redundant human emotions.") 

def get_emotions(model: str, api_key: str) -> List[str]:
    """Gets a list of 50 unique and non-redundant human emotions using the specified gpt model."""
    client = OpenAI(api_key=api_key)

    system_prompt = "Find 50 different, exclusive and unique human emotions. "\
    "For example, pick joy or happiness, pick Shame or Embarrassment, pick Envy or Jealousy, "\
    "pick Hate or disgust or hatered or Resentment. "\

    user_prompt = "Select 50 different and unique human emotions."

    try:
        completion = client.beta.chat.completions.parse(
            model= model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            response_format=EmotionsResponse
        )

        #output in the defined pydantic style
        output = completion.choices[0].message.parsed
        return output.json()
    
    except Exception as e:
        print(f"An error occurred: {e}")
        return json.dumps({})

def get_emotions_df(model, api_key):
    emotions_json = get_emotions(model, api_key)
    emotions = list(json.loads(emotions_json).values())[0]
    emotions_df = pd.DataFrame(emotions, columns = ['emotion'])
    emotions_df['emotion_id'] = emotions_df.index
    emotions_df = emotions_df[['emotion_id','emotion']]
    return emotions_df

In [264]:

#test1
#Get 100 best selling American clothing brands using the Pydantic model for the API response
class BrandResponse(BaseModel):
    brand: str = Field(description="name as a string.")
    information: str = Field(description="Brand information as a string.")
class BrandsResponse(BaseModel):
    brands: List[BrandResponse] = Field(description="A list of names and information.")

def get_brands(model: str, api_key: str) -> List[str]:
    """Get 5 best selling American clothing brands using the specified gpt model. Provide a brief information about each brand."""
    client = OpenAI(api_key=api_key)
    try:
        #Call the API to get the completion
        completion = client.beta.chat.completions.parse(
            model= model,
            messages=[
                {"role": "system", "content": "Find 5 non-redundant best selling American clothing brands."},
                {"role": "user", "content": "Give me 5 best selling American clothing brands and a brief information about each brand."}
            ],
            response_format=BrandsResponse
        )
        #output in the defined pydantic style
        output = completion.choices[0].message.parsed

        return output.json()
    
    except Exception as e:
        print(f"An error occurred: {e}")
        return json.dumps({})

In [296]:
#test2
# Embedding and getting association scores between an input and list of emotions
def emotional_association_scores(
        thing, 
        model,
        emotions, api_key
    ):

    client = OpenAI(api_key=api_key)

    Characteristic = Enum('Characteristic', dict([(emotion, emotion) for emotion in emotions]))

    class EmotionalAssociationScore(BaseModel):
        emotion: Characteristic
        score: float

    class EmotionalAssociationScores(BaseModel):
        associations: List[EmotionalAssociationScore] = Field(description="A list of emotions and associated scores")
        explanation: str = Field(description="Briefly explaining the reason behind the association scores.")

    prompt = f"Assign emotional association scores between {0} and {len(emotions)} for the provided thing. "\
    "Assign a score for each of the following emotions. Briefly, explain the reason behind the association score."\
    "Ensure the scores reflect the association strength for the specified thing. "\
    "Thing: "\
    f"{thing}"
            
    completion = client.beta.chat.completions.parse(
        model = model,
        messages=[
            {"role": "system", "content": "Be a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        response_format=EmotionalAssociationScores,
    )
    #output in the defined pydantic style
    output = completion.choices[0].message.parsed
    return thing, output.json()

In [349]:
#test get_df
def get_df(thing, model, emotions, api_key):
    gpt = emotional_association_scores(thing, model, emotions, api_key)
    data = list(json.loads(gpt[1]).values())[0]
    df = pd.DataFrame(data)
    df.rename(columns = {'score': gpt[0]}, inplace=True)
    #df[f'explanation_for_{gpt[0]}'] = json.loads(gpt[1])['explanation']
    df.set_index('emotion', inplace=True)
    brand_name = gpt[0]
    score_info = json.loads(gpt[1])['explanation']
    return {brand_name: score_info}, df
brand_scores = get_df(thing, model, emotions, api_key)


def get_dfs(things, model, emotions, api_key):
    things_scoreinfo = []
    merged_df = pd.DataFrame()
    for thing in things:
        thing_scoreinfo = get_df(thing, model, emotions, api_key)[0]
        things_scoreinfo.append(thing_scoreinfo)
        new_df = get_df(thing, model, emotions, api_key)[1]
        if merged_df.empty:
            merged_df = new_df
        else:
            merged_df = pd.merge(merged_df, new_df, left_index=True, right_index=True, how='outer')
    return things_scoreinfo, merged_df

#merged_df.reset_index(inplace= True)


def get_final_brands(model, api_key, emotions):
    brands_df = pd.DataFrame(list(json.loads(get_brands(model, api_key)).values())[0])
    brands_df.reset_index(inplace= True)
    brands_df.rename({'information':'brand_info', 'index':'brand_id'}, axis = 1, inplace = True)
    brands_df['gpt_version'] = model

    brands_scores = get_dfs(brands_df['brand'], model, emotions, api_key)
    brands_scoreinfo= brands_scores[0]
    scoreinfo_df = pd.DataFrame([(k,v) for data in brands_scoreinfo for k,v in data.items()], columns = ['brand', 'scores_info'])
    brands_df = pd.merge(brands_df, scoreinfo_df, how = 'left', on ='brand' )
    brands_df = brands_df[['brand_id','brand','brand_info', 'scores_info','gpt_version']]
    return brands_df

In [379]:
def check_emotions_exists(model, api_key, db_name, mytable):
    with sqlite3.connect(os.path.abspath(db_name)) as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (mytable,))
        if cursor.fetchone() is not None:
            print(f'Reading {mytable} from database...')
            query = f'SELECT * FROM {mytable}' 
            df = pd.read_sql_query(query, conn)
        else:
            print(f"{mytable} doesn't exist so generating one...")
            df = get_emotions_df(model, api_key)
            df.to_sql(f'{mytable}', conn, index=False)

    return df

def check_brands_exists(model, api_key, db_name, mytable, emotions, update_brand_list):
    with sqlite3.connect(os.path.abspath(db_name)) as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (mytable,))
        if cursor.fetchone() is None:
            print(f"{mytable} doesn't exist so generating one...")
            df = get_final_brands(model, api_key, emotions)
            df.to_sql(f'{mytable}', conn, index=False)
        else:
            if update_brand_list == 'no':
                print(f'Reading {mytable} from database...')
                query = f'SELECT * FROM {mytable}' 
                df = pd.read_sql_query(query, conn)
            else:
                print(f"Generating {mytable}...")
                df = get_final_brands(model, api_key, emotions)
                df.to_sql(f'{mytable}', conn, if_exists = 'replace', index=False)            
    return df

# Example usage
model = "gpt-4o-2024-08-06"
api_key=os.environ.get('OPENAI_API_KEY')
db_name = 'database.db'
update_brand_list = 'no'
number = 3
emotions_df = check_emotions_exists(model, api_key, db_name, 'emotions')
emotions = emotions_df['emotion']
brands_df = check_brands_exists(model, api_key, db_name, 'brands', emotions, update_brand_list)

emotions doesn't exist so generating one.
brands doesn't exist so generating one...


In [378]:
#check the databse tables, drop tables and check again: testing check_emotions_exists and check_brands_exists
# with sqlite3.connect(os.path.abspath(db_name)) as conn:
#     cursor = conn.cursor()
#     cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
#     tables = cursor.fetchall()
#     table_names = [table[0] for table in tables]
#     print("Tables in the database:", table_names)
#     for table in table_names:
#         query = f'SELECT * FROM {table}' 
#         table = pd.read_sql_query(query, conn)
#         print(table.head())
        
# # Drop all tables and check again the above works
# with sqlite3.connect(os.path.abspath(db_name)) as conn:
#     cursor = conn.cursor()
#     cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
#     tables = cursor.fetchall()
#     table_names = [table[0] for table in tables]
#     print("Tables in the database:", table_names)
#     for table in table_names:
#         cursor.execute(f"DROP TABLE IF EXISTS {table};")
#         print(f"Table {table} dropped")
#     conn.commit()

Tables in the database: []


In [ ]:

def get_similarity(df, dfs, number):
    dfs.set_index('emotion', inplace=True)

    similarities = dict()
    df_cosine = df[[col for col in df.columns if 'explanation' not in col]]
    # Reshape Series to 2D array (required by cosine_similarity)
    s1 = df_cosine.values.reshape(1, -1)

    for col in list(dfs.columns):
        dfs_cosine = dfs[[col for col in dfs.columns if 'explanation' not in col and 'gpt' not in col]]
        s2= dfs_cosine[col].values.reshape(1, -1)
        cosine_sim = cosine_similarity(s1, s2)
        similarities[col]= cosine_sim[0][0]

    sorted_dict = dict(sorted(similarities.items(), key=lambda item: item[1], reverse = True))

    # Get the top number of recommendations based on similarity
    recommendations = list(dict(islice(sorted_dict.items(), number)).keys())
    return recommendations

In [ ]:
df = get_df(thing, model, emotions, api_key)
df_cleaned = df.dropna(axis=1)

dfs = get_dfs(things, model, emotions, api_key)
dfs_cleaned = dfs.dropna(axis=1)


result = get_similarity(df_cleaned, dfs_cleaned, number)
result

In [382]:
emotions_df

,emotion_id,emotion
0,0,Joy
1,1,Sorrow
2,2,Fear
3,3,Disgust
4,4,Surprise
5,5,Trust
6,6,Anticipation
7,7,Anger
8,8,Contentment
9,9,Excitement


In [383]:
brands_df

,brand_id,brand,brand_info,scores_info,gpt_version
0,0,Nike,Founded in 1964 and headquartered in Beaverton...,"Nike is often associated with joy, pride, and ...",gpt-4o-2024-08-06
1,1,Ralph Lauren,"Established in 1967, Ralph Lauren is synonymou...",Ralph Lauren is primarily associated with luxu...,gpt-4o-2024-08-06
2,2,Levi's,"Founded in 1853 in San Francisco, California, ...","Levi’s, as a brand, evokes varying emotional a...",gpt-4o-2024-08-06
3,3,Tommy Hilfiger,"Started in 1985, Tommy Hilfiger is known for i...","Tommy Hilfiger, as a global brand, is often as...",gpt-4o-2024-08-06
4,4,Under Armour,"Founded in 1996 and based in Baltimore, Maryla...","Under Armour is a popular athletic brand, and ...",gpt-4o-2024-08-06
